# Global maps for the report

This is a bit messy, but there is a bunch of code producing maps for the technical report.

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import sys, os
from oggm import utils
import csv
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
from matplotlib.image import imread
import salem

In [ ]:
# go down from rgi7_scripts/workflow
data_dir = '../../../../rgi7_data/'

final_dir = os.path.join(data_dir, 'rgi7_final')

In [ ]:
user_guide_dir = '../../../../rgi_user_guide/'

In [ ]:
tmp_dir = data_dir + '/tmp_plots/'

## Read RGI6 and RGI7 tables 

In [ ]:
df_rgi7g = pd.read_csv(os.path.join(final_dir, 'global_files', 'attributes', 'RGI2000-v7.0-G-global-attributes.csv.zip'), 
                       index_col=0,
                       compression='zip',
                       dtype={'o1region': str})

In [ ]:
df_rgi6g = pd.read_hdf(utils.file_downloader('https://cluster.klima.uni-bremen.de/~oggm/rgi/rgi62_stats.h5'))
df_rgi6g = df_rgi6g.loc[df_rgi6g.Connect != 2]

In [ ]:
df_rgi7c = pd.read_csv(os.path.join(final_dir, 'global_files', 'attributes', 'RGI2000-v7.0-C-global-attributes.csv.zip'), 
                       index_col=0,
                       compression='zip',
                       dtype={'o1region': str})

In [ ]:
np.testing.assert_allclose(df_rgi7g.area_km2.sum(), df_rgi7c.area_km2.sum())

In [ ]:
df_reg_o1 = gpd.read_file('zip://' + final_dir + '/RGI2000-v7.0-regions.zip/RGI2000-v7.0-o1regions.shp')
df_reg_o2 = gpd.read_file('zip://' + final_dir + '/RGI2000-v7.0-regions.zip/RGI2000-v7.0-o2regions.shp')

In [ ]:
df_reg_o1['min_x'] = [g.bounds[0] for g in df_reg_o1.geometry]
df_reg_o1['max_x'] = [g.bounds[2] for g in df_reg_o1.geometry]
df_reg_o1['min_y'] = [g.bounds[1] for g in df_reg_o1.geometry]
df_reg_o1['max_y'] = [g.bounds[3] for g in df_reg_o1.geometry]

## Map 

In [ ]:
rgi6 = df_rgi6g.copy()
rgi7 = df_rgi7g.copy()

In [ ]:
%config InlineBackend.figure_format = 'retina'
%alias convert convert

In [ ]:
bck_img = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'

In [ ]:
fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())


ax.imshow(imread(bck_img), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])

handles = []

df_reg_o1.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

s = rgi7.loc[rgi7.is_rgi6 == 1]
ax.scatter(s.cenlon.values, s.cenlat.values, color='C0', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C0', label=f"Same as RGI 6.0"))  #  - N={len(s)}, A={int(s['area'].sum()*1e-6)} km²

s = rgi7.loc[rgi7.is_rgi6 == 0]
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"New in RGI 7.0"))

df_reg_o1.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df_reg_o1.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);
plt.tight_layout();

plt.savefig(tmp_dir + '/global_map_wrgi6.png', dpi=150, bbox_inches='tight')

In [ ]:
tmp_dir, user_guide_dir

In [ ]:
! convert ../../../../rgi7_data/tmp_plots/global_map_wrgi6.png -quality 85 ../../../../rgi_user_guide/docs/img/global_stats/global_map_wrgi6.jpeg

In [ ]:
! convert ../../../../rgi7_data/tmp_plots/global_map_wrgi6.png -resize 900 -quality 75  ../../../../rgi_user_guide/docs/img/global_stats/global_map_wrgi6_small.jpeg

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
ax.imshow(imread(bck_img), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])
handles = []

df_reg_o1.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

s = rgi7
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"RGI 7.0 glacier outlines"))

df_reg_o1.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df_reg_o1.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);
plt.tight_layout();

plt.savefig(tmp_dir + '/global_map.png', dpi=150, bbox_inches='tight')

In [ ]:
! convert ../../../../rgi7_data/tmp_plots/global_map.png -quality 85 ../../../../rgi_user_guide/docs/img/global_stats/global_map.jpeg

In [ ]:
! convert ../../../../rgi7_data/tmp_plots/global_map.png -resize 900 -quality 75  ../../../../rgi_user_guide/docs/img/global_stats/global_map_small.jpeg

In [ ]:
fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
ax.imshow(imread(bck_img), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])
handles = []

df_reg_o1.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

for cat, col, name in zip([0, 1, 2, 3], ['royalblue', 'C2', 'C1', 'C3'], ['No evidence', 'Possible', 'Probable', 'Observed']):
    s = rgi7.loc[rgi7.surge_type == cat]
    if name == 'No evidence':
        ax.scatter(s.cenlon.values, s.cenlat.values, fc=col, ec='None', s=1, transform=ccrs.PlateCarree(), linewidth=0);
    else:
        ax.scatter(s.cenlon.values, s.cenlat.values, fc=col, ec='k', s=15, transform=ccrs.PlateCarree(), linewidth=0.5);
        
    handles.append(mpatches.Patch(facecolor=col, label=name))


df_reg_o1.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df_reg_o1.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


legend = ax.legend(handles=handles, loc='center left', fontsize=fs, title='Surge type');
plt.setp(legend.get_title(),fontsize=fs+2)

plt.tight_layout();

plt.savefig(tmp_dir + '/global_map_surge.png', dpi=150, bbox_inches='tight')

In [ ]:
! convert ../../../../rgi7_data/tmp_plots/global_map_surge.png -quality 85 ../../../../rgi_user_guide/docs/img/global_stats/global_map_surge.jpeg

In [ ]:
! convert ../../../../rgi7_data/tmp_plots/global_map_surge.png -resize 900 -quality 75  ../../../../rgi_user_guide/docs/img/global_stats/global_map_surge_small.jpeg

In [ ]:
fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
ax.imshow(imread(bck_img), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])
handles = []

df_reg_o1.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

for cat, col, name in zip([9, 1], ['royalblue', 'C3'], ['Not assigned', 'Marine-terminating']):
    s = rgi7.loc[rgi7.term_type == cat]
    if name == 'Not assigned':
        ax.scatter(s.cenlon.values, s.cenlat.values, fc=col, ec='None', s=1, transform=ccrs.PlateCarree(), linewidth=0);
    else:
        ax.scatter(s.cenlon.values, s.cenlat.values, fc=col, ec='k', s=15, transform=ccrs.PlateCarree(), linewidth=0.5);
        
    handles.append(mpatches.Patch(facecolor=col, label=name))


df_reg_o1.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df_reg_o1.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


legend = ax.legend(handles=handles, loc='center left', fontsize=fs, title='Terminus type');
plt.setp(legend.get_title(),fontsize=fs+2)

plt.tight_layout();

plt.savefig(tmp_dir + '/global_map_term_type.png', dpi=150, bbox_inches='tight')

In [ ]:
! convert ../../../../rgi7_data/tmp_plots/global_map_term_type.png -quality 85 ../../../../rgi_user_guide/docs/img/global_stats/global_map_term_type.jpeg

In [ ]:
! convert ../../../../rgi7_data/tmp_plots/global_map_term_type.png -resize 900 -quality 75  ../../../../rgi_user_guide/docs/img/global_stats/global_map_term_type_small.jpeg

In [ ]:
fac = 1.3
f = plt.figure(figsize=(7*fac, 7*fac))
ax = plt.axes(projection=ccrs.Orthographic(-20, 45))

ax.imshow(imread(bck_img), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90]);

s = rgi7
handles = []
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=0.1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"Glacier outlines"))
ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();